In [2]:
import os
import pandas as pd 

dataframe = pd.read_csv('train.csv')

for index in dataframe.columns:
    print('Missing value: ',dataframe[index].isnull().sum())
    print(dataframe[index].describe())
    print()


Missing value:  0
count                      416962
unique                     173711
top       2019-10-02 20:37:56 UTC
freq                          102
Name: event_time, dtype: object

Missing value:  0
count     416962
unique         4
top         view
freq      168854
Name: event_type, dtype: object

Missing value:  0
count    4.169620e+05
mean     5.498023e+06
std      1.268676e+06
min      3.752000e+03
25%      5.729011e+06
50%      5.809809e+06
75%      5.850305e+06
max      5.892800e+06
Name: product_id, dtype: float64

Missing value:  0
count    4.169620e+05
mean     1.556230e+18
std      1.644289e+17
min      1.490000e+18
25%      1.490000e+18
50%      1.490000e+18
75%      1.490000e+18
max      2.200000e+18
Name: category_id, dtype: float64

Missing value:  410798
count                              6164
unique                               10
top       appliances.environment.vacuum
freq                               2982
Name: category_code, dtype: object

Missing value:  16

In [3]:
dataframe['product_id'] = dataframe['product_id'].astype('object')
print(dataframe['product_id'].dtype)
print(dataframe['product_id'].describe())

object
count      416962
unique      32734
top       5892179
freq         1063
Name: product_id, dtype: int64


In [4]:
dataframe.fillna(value={'brand':'null'},inplace=True)
print(dataframe['brand'].isnull().sum())


0


In [4]:
import datetime

index_len = len(dataframe.index)
date_list = []

for i in range(index_len):
    date_str = dataframe['event_time'][i]
    date_list.append(datetime.datetime.strptime(date_str[0:19],'%Y-%m-%d %H:%M:%S'))

dataframe['event_time'] = pd.DataFrame(date_list)
print(dataframe['event_time'].dtype)
print(dataframe['event_time'].head())

datetime64[ns]
0   2019-10-01 00:00:00
1   2019-10-01 00:00:03
2   2019-10-01 00:00:07
3   2019-10-01 00:02:32
4   2019-10-01 00:02:40
Name: event_time, dtype: datetime64[ns]


In [5]:
print(dataframe[dataframe['price'] < 0].index)
dataframe = dataframe.drop(dataframe[dataframe['price'] < 0].index)
print(dataframe[dataframe['price'] < 0].index)

Int64Index([63196, 165348], dtype='int64')
Int64Index([], dtype='int64')


In [6]:
print(dataframe.dtypes)

event_time       datetime64[ns]
event_type               object
product_id               object
category_id             float64
category_code            object
brand                    object
price                   float64
user_id                   int64
user_session             object
dtype: object


In [8]:
print(dataframe.head())

           event_time event_type product_id   category_id category_code  \
0 2019-10-01 00:00:00       cart    5773203  1.490000e+18           NaN   
1 2019-10-01 00:00:03       cart    5773353  1.490000e+18           NaN   
2 2019-10-01 00:00:07       cart    5723490  1.490000e+18           NaN   
3 2019-10-01 00:02:32       cart    5857283  1.490000e+18           NaN   
4 2019-10-01 00:02:40       cart    5723523  1.490000e+18           NaN   

    brand  price  user_id                          user_session  
0  runail   2.62        1  26dd6e6e-4dac-4778-8d2c-92e149dab885  
1  runail   2.62        1  26dd6e6e-4dac-4778-8d2c-92e149dab885  
2  runail   2.62        1  26dd6e6e-4dac-4778-8d2c-92e149dab885  
3  runail   2.62        1  26dd6e6e-4dac-4778-8d2c-92e149dab885  
4  runail   2.62        1  26dd6e6e-4dac-4778-8d2c-92e149dab885  


任务分工说明：  
1. 产品，价格，品牌是否重叠？重叠度有多高 （王）  
2. 价格（连续变量）离散化以及brand空值处理  （王） 注：离群点的存在，合理划分区间  
3. 对购买之后的记录进行删除操作 （王）  
4. 剔除无购买行为的用户  （王）  
5. 构建品牌-价格字典 （张）  
6. 设计测试评价指标 （张）  
7. 设计个人行为模型，要考虑event,price,brand等变量，并且对event变量做好处理，通过购买产品进行监督训练 （邢）  
8. 考虑如何合成公共字典和个人行为模型，以概率的形式输出不同产品购买的可能性 （邢）  
  
具体要求：  
王：返回处理结束后的csv文件，并对重叠度进行基本描述  
张：基于新的数据集构建字典，返回字典格式的数据，其中字典中的值为对应产品的频数，实现测试评价指标的代码，输入为预测的概率序列（从高到低），输出为每个产品的召回率和整体的平均倒数排名  
邢：实现根据输入变量预测产品输出概率的模型，并设计好训练模式，实现将公共字典中的产品概率与浏览产品的预测概率结合的最终产品概率预测算法  


In [11]:
# import dataset after preprocess by wang
train_data = pd.read_csv('train_clear.csv')
print(train_data.dtypes)
print(train_data['brand'].isnull().sum())
print(train_data['price'].value_counts())
train_data.fillna(value={'brand':'null'},inplace=True)
print(train_data['brand'].isnull().sum())
print()
test_data = pd.read_csv('test_clear.csv')
print(test_data.dtypes)
print(test_data['brand'].isnull().sum())
print(test_data['price'].value_counts())
test_data.fillna(value={'brand':'null'},inplace=True)
print(test_data['brand'].isnull().sum())
test_data['price'] = test_data['price'].astype('int64')
print(test_data['price'].dtype)

event_time        object
event_type        object
product_id         int64
category_id      float64
category_code     object
brand             object
price              int64
user_id            int64
user_session      object
dtype: object
44357
1    30177
2    27638
3    21483
4    18852
5    14379
Name: price, dtype: int64
0

event_time        object
event_type        object
product_id         int64
category_id      float64
category_code     object
brand             object
price              int64
user_id            int64
user_session      object
dtype: object
426
1    342
2    328
4    195
3    182
5    167
Name: price, dtype: int64
0
int64


In [14]:
print(len(train_data.index))
print(len(test_data.index))
print(len(train_data['user_id'].unique()))
print(len(test_data['user_id'].unique()))

112529
1214
2636
49


In [18]:
import warnings

warnings.filterwarnings('ignore')
#build a dictionary based on brand and price
diction = {}
#print(len(dataframe['brand'].unique()))
a = train_data['brand'].value_counts()
price_level = [1,2,3,4,5] #Discretization
for key in a.keys():
    diction[key] = {}
    for val in price_level:
        diction[key][val] = {}




# basic assumption: purchase event is the most related event, frequency contributes little.
user_ids = train_data['user_id'].value_counts()

for user_id, count in enumerate(user_ids):
    record = train_data[train_data['user_id'] == user_id][train_data['event_type'] == 'purchase']
    #remove duplicate
    update_record = record.drop_duplicates(subset=['product_id'],keep='first',inplace=False)
    
    for i in update_record.index:
        product_id = update_record.loc[i,'product_id']
        brand = update_record.loc[i,'brand']
        price = update_record.loc[i,'price']
        if product_id not in diction[brand][price]:
            diction[brand][price][product_id] = 1
        else:
            diction[brand][price][product_id] += 1
        





[('null', {1: {5853646: 1, 5802428: 3, 5688124: 4, 5825598: 3, 5549834: 4, 5586026: 1, 5626878: 2, 5686225: 2, 5687151: 5, 5693501: 4, 5711069: 1, 5712452: 1, 5865795: 1, 5875317: 3, 5842120: 1, 5686080: 1, 5817702: 5, 5802427: 1, 5687478: 2, 5733176: 2, 5742402: 1, 5697283: 1, 5742401: 3, 5585656: 3, 5867922: 1, 5813882: 2, 5817690: 1, 5865800: 2, 5762152: 1, 5686925: 2, 5844301: 2, 5843836: 5, 5842162: 2, 5843595: 1, 5843591: 1, 5843588: 1, 5843579: 1, 5843564: 1, 5840357: 1, 9822: 1, 9825: 1, 29369: 1, 35344: 1, 5887011: 1, 5698860: 1, 5815662: 7, 5862555: 1, 5886746: 1, 5886777: 1, 5853633: 1, 5866166: 2, 5884230: 1, 5664665: 2, 5717245: 1, 5803297: 1, 5815653: 4, 5664641: 1, 5884238: 1, 5729864: 5, 5809721: 1, 5790685: 2, 5784898: 1, 5845360: 1, 5565822: 2, 5815656: 2, 5585658: 1, 5842134: 1, 5680394: 1, 5890987: 1, 5873642: 4, 5723225: 4, 5710108: 2, 5710106: 1, 5687138: 2, 5687135: 1, 5744388: 1, 5744425: 1, 5747361: 5, 5843596: 2, 5841721: 3, 5687132: 2, 5687131: 4, 5695546: 1,

In [20]:
print(diction['masura'])

{1: {5859462: 3, 5826988: 2, 5813058: 3, 5826989: 1, 5797965: 1, 5859468: 1, 5788524: 1, 5796976: 1, 5796983: 2, 5797967: 1, 5801770: 1, 5804167: 1, 5804168: 1, 5813060: 1, 5813068: 1, 5826996: 1, 5867965: 1, 5774877: 1, 5774872: 1, 5830548: 2, 5774871: 1, 5788522: 1, 5794153: 1, 5796995: 1, 5774893: 1, 5830534: 1, 5603537: 1, 5813483: 2, 5813064: 1, 5859478: 2, 5830561: 1, 5813482: 1, 5839699: 1, 5839706: 1, 5839703: 1, 5839702: 1, 5839697: 1, 5758509: 1, 5774897: 1, 5796998: 1, 5794155: 1}, 2: {5859410: 1, 5859411: 1, 5859413: 1, 5839677: 2, 5859407: 1, 5839667: 1, 5859420: 1, 5839676: 1, 5839672: 2, 5859402: 3, 5683351: 1, 5839656: 1}, 3: {5883344: 1, 5839690: 1, 5746386: 1, 5774863: 1, 5853684: 1}, 4: {5627420: 1, 5624537: 1, 5747404: 1}, 5: {5774857: 1, 27764: 1}}


In [35]:
#build a judgement calculation method
predict_1 = {5802428: 0.8, 5688124: 0.6, 5739989: 0.4, 5773353: 0.2}
truth_1 = [5688124, 5773353, 5385]
predict_2 = {5688124:0.7, 5385:0.5, 5555:0.9}
truth_2 = [5385,5688124]
predict = [predict_1,predict_2]
truth = [truth_1,truth_2]
#MRR
def MRR(p,t):
    Q = len(p)
    mrr = 0
    for i in range(Q):
        temp_p = p[i]
        temp_t = t[i]
        p_value = sorted(temp_p.items(),key= lambda item:item[1], reverse=True)
        p_len = len(p_value)
        find = False
        for j in range(p_len):
            k,v = p_value[j]
            if k in temp_t:
                rank = 1/(j+1)
                find = True
                break

        if find == False:
            #optimistic : the recommend algorithm will find truth product next.
            rank = 1/(p_len+1)

        mrr += rank
    mrr /= Q
    return mrr

print(MRR(predict,truth))

#Inclusion Rate: a numerical identifier to measure to what extent truth product is covered by predict product
def Inclusion(p,t):
    Q = len(p)
    inclusion = 0
    for i in range(Q):
        temp_p = p[i]
        temp_t = t[i]
        score = 0
        for member in temp_t:
            if member in temp_p.keys():
                score += temp_p[member]
        
        score /= len(temp_t)
        inclusion += score
    
    inclusion /= Q
    return inclusion

print(Inclusion(predict,truth))


0.5
0.43333333333333335
